In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ.setdefault('MINIO_BUCKET', 'ae-de-play')

import duckdb
from src.settings import settings

In [ ]:
conn = duckdb.connect(settings.DUCKDB_DATABASE)

In [ ]:
conn.sql("""
SELECT * FROM public.cchain__climate_atmosphere
ORDER BY uuid
LIMIT 20
""").pl()